# Construir Agentes de IA com Memória Persistente usando Cognee

Este notebook demonstra como criar agentes de IA inteligentes com capacidades avançadas de memória utilizando [**cognee**](https://www.cognee.ai/) - uma memória de IA de código aberto que combina grafos de conhecimento, pesquisa semântica e gestão de sessões para criar sistemas de IA com consciência de contexto.

## 🎯 Objetivos de Aprendizagem

Ao final deste tutorial, você entenderá como:
- **Construir Grafos de Conhecimento Baseados em Embeddings**: Transformar texto não estruturado em conhecimento estruturado e consultável
- **Implementar Memória de Sessão**: Criar conversas de múltiplas interações com retenção automática de contexto
- **Persistir Conversas**: Armazenar opcionalmente interações importantes em memória de longo prazo para referência futura
- **Consultar Usando Linguagem Natural**: Acessar e aproveitar o contexto histórico em novas conversas
- **Visualizar Memória**: Explorar as relações no grafo de conhecimento do seu agente


## 🏗️ O Que Irá Construir

Neste tutorial, vamos criar um **Assistente de Programação** com memória persistente que:

### 1. **Construção de Base de Conhecimento**
   - Recolhe informações sobre o perfil e a experiência do programador
   - Processa princípios e boas práticas de programação em Python
   - Armazena conversas históricas entre programadores e assistentes de IA

### 2. **Conversas Sensíveis ao Contexto**
   - Mantém o contexto ao longo de várias perguntas na mesma sessão
   - Armazena automaticamente cada par de pergunta/resposta para recuperação eficiente
   - Fornece respostas coerentes e contextuais com base no histórico da conversa

### 3. **Memória de Longo Prazo**
   - Persiste conversas importantes numa memória de longo prazo
   - Recupera memórias relevantes da base de conhecimento e de sessões passadas para informar novas interações
   - Constrói uma base de conhecimento crescente que melhora ao longo do tempo

### 4. **Recuperação Inteligente de Memória**
   - Utiliza pesquisa semântica baseada em grafos para encontrar informações relevantes em todo o conhecimento armazenado
   - Filtra pesquisas por subgrupos de dados (informações do programador vs. princípios)
   - Combina múltiplas fontes de dados para fornecer respostas abrangentes


## 📋 Pré-requisitos e Configuração

### Requisitos do Sistema

Antes de começar, certifique-se de ter:

1. **Ambiente Python**
   - Python 3.9 ou superior
   - Ambiente virtual (recomendado)
   
2. **Cache Redis** (Necessário para Gestão de Sessões)
   - Redis local: `docker run -d -p 6379:6379 redis`
   - Ou utilize um serviço Redis gerido
   
3. **Acesso à API LLM**
   - Chave de API OpenAI ou outros fornecedores (consulte [documentação](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Configuração de Base de Dados**
   - Nenhuma configuração necessária por padrão. O Cognee utiliza bases de dados baseadas em ficheiros (LanceDB e Kuzu)
   - Opcionalmente, pode configurar o Azure AI Search como um repositório vetorial (consulte [documentação](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configuração do Ambiente

Crie um ficheiro `.env` no diretório do seu projeto com as seguintes variáveis:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Compreendendo a Arquitetura de Memória do Cognee

### Como o Cognee Funciona

O Cognee oferece um sistema de memória sofisticado que vai além do armazenamento simples de chave-valor:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Componentes Principais:

1. **Knowledge Graph**: Armazena entidades, relações e conexões semânticas
2. **Vector Embeddings**: Permite pesquisa semântica em todas as informações armazenadas
3. **Session Cache**: Mantém o contexto da conversa dentro e entre sessões
4. **NodeSets**: Organizam os dados em categorias lógicas para recuperação direcionada

### Tipos de Memória Neste Tutorial:

- **Memória Persistente**: Armazenamento de longo prazo no knowledge graph
- **Memória de Sessão**: Contexto temporário de conversação no cache Redis
- **Memória Semântica**: Pesquisa baseada em similaridade vetorial em todos os dados


## 📦 Instalar Pacotes Necessários

Instale o Cognee com suporte ao Redis para gestão de sessões:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializar Ambiente e Carregar Bibliotecas

Certifique-se de que:
1. O Redis está em execução (por exemplo, via Docker: `docker run -d -p 6379:6379 redis`)
2. As variáveis de ambiente estão definidas antes de importar os módulos de cache
3. Se necessário, reinicie o kernel e execute as células na ordem correta

A célula seguinte irá:
1. Carregar variáveis de ambiente a partir do ficheiro `.env`
2. Configurar o Cognee com as suas definições de LLM
3. Ativar o cache para gestão de sessões
4. Validar que todos os componentes estão devidamente conectados


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configurar Diretórios de Armazenamento

O Cognee utiliza dois diretórios separados para as suas operações:
- **Data Root**: Armazena documentos ingeridos e dados processados
- **System Root**: Contém a base de dados do grafo de conhecimento e metadados do sistema

Vamos criar diretórios isolados para este tutorial da seguinte forma:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Repor o Estado da Memória

Antes de começarmos a construir o nosso sistema de memória, vamos garantir que estamos a começar do zero.

> 💡 **Dica**: Pode ignorar este passo se quiser preservar memórias existentes das suas execuções anteriores ao usar este notebook mais tarde.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Parte 1: Construir a Base de Conhecimento

### Fontes de Dados para o Nosso Assistente de Desenvolvedor

Vamos integrar três tipos de dados para criar uma base de conhecimento abrangente:

1. **Perfil do Desenvolvedor**: Experiência pessoal e formação técnica  
2. **Melhores Práticas em Python**: O Zen do Python com diretrizes práticas  
3. **Conversas Históricas**: Sessões de perguntas e respostas anteriores entre desenvolvedores e assistentes de IA  

Estes dados diversificados permitem que o nosso agente:  
- Compreenda o contexto técnico do utilizador  
- Aplique as melhores práticas nas recomendações  
- Aprenda com interações bem-sucedidas anteriores  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Processar Dados em um Grafo de Conhecimento

Agora vamos transformar o nosso texto bruto em uma memória estruturada. Este processo:

1. **Adiciona dados aos NodeSets**: Organiza informações em categorias lógicas
   - `developer_data`: Perfil do desenvolvedor e conversas
   - `principles_data`: Melhores práticas e diretrizes de Python

2. **Executa o Cognify Pipeline**: Extrai entidades, relacionamentos e cria embeddings
   - Identifica conceitos-chave
   - Cria conexões semânticas entre informações relacionadas
   - Gera embeddings vetoriais

Isto pode levar alguns momentos enquanto o LLM processa o texto e constrói a estrutura do grafo:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualizar o Grafo de Conhecimento

Vamos explorar a estrutura do nosso grafo de conhecimento. A visualização mostra:
- **Nós**: Entidades extraídas do texto (conceitos, tecnologias, pessoas)
- **Arestas**: Relações e conexões entre as entidades
- **Clusters**: Conceitos relacionados agrupados por similaridade semântica

Abra o ficheiro HTML gerado no seu navegador para explorar o grafo de forma interativa:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Enriquecer a Memória com Memify

A função `memify()` analisa o grafo de conhecimento e gera regras inteligentes sobre os dados. Este processo:
- Identifica padrões e melhores práticas
- Cria diretrizes acionáveis com base no conteúdo
- Estabelece relações entre diferentes áreas de conhecimento

Estas regras ajudam o agente a tomar decisões mais informadas ao responder a perguntas. Capturar uma segunda visualização ajuda a comparar como o grafo se densifica após ser enriquecido.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Parte 2: Recuperação Inteligente de Memória

### Demonstração 1: Integração de Conhecimento entre Documentos

Agora que o nosso grafo de conhecimento está construído, vamos testar como o Cognee combina informações de várias fontes para responder a perguntas complexas.

A primeira consulta demonstra:
- **Compreensão semântica**: Encontrar conceitos relevantes mesmo quando não mencionados explicitamente
- **Referenciamento cruzado**: Combinar o perfil do desenvolvedor com princípios de Python
- **Raciocínio contextual**: Aplicar as melhores práticas a implementações específicas

### Demonstração 2: Pesquisa Filtrada com NodeSets

A segunda consulta mostra como direcionar subconjuntos específicos do grafo de conhecimento:
- Utiliza o parâmetro `node_name` para pesquisar apenas dentro de `principles_data`
- Fornece respostas focadas de um domínio de conhecimento específico
- Útil quando precisa de informações específicas de um domínio


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Parte 3: Configuração da Gestão de Sessões

### Ativar Memória de Conversação

A gestão de sessões é essencial para manter o contexto ao longo de várias interações. Aqui iremos:

1. **Inicializar o Contexto do Utilizador**: Criar ou recuperar um perfil de utilizador para rastreamento da sessão  
2. **Configurar o Motor de Cache**: Ligar ao Redis para armazenar o histórico de conversas  
3. **Ativar Variáveis de Sessão**: Configurar variáveis de contexto que persistem entre consultas  

> ⚠️ **Importante**: Isto requer que o Redis esteja em execução e `CACHING=true` no seu ambiente


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Função Auxiliar: Ver Histórico de Sessão

Esta função utilitária permite-nos inspecionar o histórico de conversação armazenado no Redis. É útil para:
- Depurar a gestão de sessões
- Verificar se as conversas estão a ser armazenadas em cache
- Compreender que contexto está disponível para o agente


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sessão 1: Laboratório de Suporte Assíncrono — Primeira Pergunta

Inicie a sessão `async-support-lab` perguntando sobre padrões de asyncio compatíveis com telemetria para um web scraper em larga escala. O gráfico já conhece asyncio, aiohttp e práticas de monitorização, então a resposta deve refletir conversas anteriores enquanto adapta a resposta à nova pergunta.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspecionar a Memória da Sessão 1 Após a Primeira Troca

Executar `show_history(session_1)` imediatamente após a pergunta inicial confirma que o Cognee gravou tanto o prompt quanto a resposta no Redis. Deve ver uma entrada com a orientação de concorrência.


In [ ]:
await show_history(session_1)

## Sessão 1: Acompanhamento sobre Modelos de Dados

A seguir, perguntamos: "Quando devo escolher dataclasses em vez de Pydantic?" usando o mesmo ID de sessão. Cognee deve combinar os princípios do Python com conversas anteriores sobre FastAPI para fornecer conselhos detalhados—demonstrando que o contexto é mantido dentro de uma sessão nomeada.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirmar que o Histórico da Sessão 1 Contém Ambos os Turnos

Outra chamada a `show_history(session_1)` deve revelar duas entradas de Pergunta e Resposta. Isto corresponde ao passo de "replay de memória" do laboratório Mem0 e prova que turnos adicionais prolongam a mesma transcrição.


In [ ]:
await show_history(session_1)

## Sessão 2: Tópico de Revisão de Design — Nova Sessão

Para demonstrar isolamento entre tópicos, iniciamos `design-review-session` e pedimos orientações de registo para revisões de incidentes. Embora a base de conhecimento subjacente seja a mesma, o novo ID de sessão mantém as transcrições separadas.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Revisão da Sessão 2 História

`show_history(session_2)` deve listar apenas o par de solicitação/resposta da revisão de design. Compare com a Sessão 1 para destacar como o Cognee mantém transcrições independentes enquanto reutiliza o grafo de conhecimento compartilhado.


In [ ]:
await show_history(session_2)

## Resumo

Parabéns! Acabou de dar ao seu assistente de programação uma camada de memória de longo prazo alimentada pelo Cognee.

Neste tutorial, pegou conteúdo bruto de desenvolvedores (código, documentos, conversas) e transformou-o numa memória em grafo + vetor que o seu agente pode pesquisar, raciocinar e melhorar continuamente.

O que Aprendeu

1. **De texto bruto a memória de IA**: Como o Cognee processa dados não estruturados e os transforma numa memória inteligente e pesquisável, utilizando uma arquitetura combinada de vetor + grafo de conhecimento.

2. **Enriquecimento de grafos com memify**: Como ir além da criação básica de grafos e usar o memify para adicionar factos derivados e relações mais ricas ao seu grafo existente.

3. **Múltiplas estratégias de pesquisa**: Como consultar a memória com diferentes tipos de pesquisa (Q&A com consciência de grafo, conclusão ao estilo RAG, insights, fragmentos brutos, pesquisa de código, etc.), dependendo das necessidades do seu agente.

4. **Exploração visual**: Como inspecionar e depurar o que o Cognee construiu utilizando visualizações de grafos e a interface do Cognee, para que possa realmente ver como o conhecimento está estruturado.

5. **Memória sensível à sessão**: Como combinar o contexto de cada sessão com uma memória semântica persistente, permitindo que os agentes se lembrem entre execuções sem partilhar informações entre utilizadores.


## Principais Conclusões
1. Memória como um Grafo de Conhecimento suportado por Embeddings

   - **Compreensão estruturada**: O Cognee combina um armazenamento vetorial e um armazenamento em grafo, permitindo que os seus dados sejam pesquisáveis tanto por significado quanto conectados por relações. O Cognee utiliza bases de dados baseadas em ficheiros por padrão (LanceDB para armazenamento vetorial, Kuzu para base de dados em grafo).

   - **Recuperação consciente de relações**: As respostas podem ser fundamentadas não apenas em "texto semelhante", mas também em como as entidades se relacionam.

   - **Memória viva**: A camada de memória evolui, cresce e permanece consultável como um grafo conectado.

2. Modos de Pesquisa e Raciocínio
   - **Recuperação híbrida**: A pesquisa combina similaridade vetorial, estrutura de grafo e raciocínio de LLM, desde a pesquisa de fragmentos brutos até respostas a perguntas conscientes do grafo.

   - **Ajustar o modo à tarefa**: Utilize modos de estilo de conclusão quando quiser respostas em linguagem natural, e modos de fragmento/resumo/grafo quando o seu agente precisar de contexto bruto ou para conduzir o seu próprio raciocínio.

3. Agentes Personalizados e Conscientes da Sessão
   - **Contexto da sessão + memória de longo prazo**: O Cognee mantém o contexto de "fio" de curto prazo separado da memória de longo prazo, ao nível do utilizador ou da organização.

## Aplicações no Mundo Real

1. **Agentes de IA Verticais**

   Utilize o padrão deste notebook para criar copilotos inteligentes em domínios específicos que utilizam o Cognee como núcleo de recuperação e raciocínio:

- **Copilotos para programadores**: Revisão de código, análise de incidentes e assistentes de arquitetura que percorrem código, APIs, documentos de design e tickets como um único grafo de memória.

- **Copilotos voltados para o cliente**: Agentes de suporte ou sucesso que extraem informações de documentos de produto, FAQs, notas de CRM e tickets anteriores com recuperação consciente do grafo e respostas citadas.

- **Copilotos especialistas internos**: Assistentes de política, jurídico ou segurança que raciocinam sobre regras interconectadas, diretrizes e decisões históricas em vez de PDFs isolados.

   O Cognee posiciona-se explicitamente como uma memória persistente e precisa para agentes de IA, fornecendo um grafo de conhecimento vivo que se integra ao seu agente e substitui combinações ad hoc de armazenamentos vetoriais e código de grafo personalizado.

2. **Unificar Silos de Dados numa Única Memória**

   A mesma abordagem também ajuda a construir uma camada de memória unificada a partir de fontes dispersas:

- **De silos para um único grafo**: Ingestão de dados estruturados (por exemplo, bases de dados) e não estruturados (por exemplo, documentos, chats) num único grafo suportado por embeddings, em vez de índices separados para cada sistema.

- **Raciocínio entre fontes com citações**: Realize raciocínios em múltiplas etapas sobre tudo—"junte" registos, métricas e documentos através do grafo—e ainda assim devolva respostas fundamentadas com proveniência.

- **Hubs de conhecimento**: Para domínios como banca ou educação, o Cognee já é utilizado para unificar PDFs, sistemas internos e dados de aplicações num único grafo de conhecimento com vetores, permitindo que os agentes respondam a perguntas com contexto preciso e citado.

## Próximos Passos

Já implementou o ciclo de memória principal. Aqui estão extensões naturais que pode experimentar por conta própria (consulte a [documentação do Cognee](https://docs.cognee.ai/) para mais detalhes):

1. **Experimente a consciência temporal**: Ative o "temporal cognify" para extrair eventos e marcas temporais do texto.

2. **Introduza raciocínio orientado por ontologia**: Defina uma ontologia OWL para o seu domínio. Utilize o suporte de ontologia do Cognee para que as entidades e relações extraídas sejam fundamentadas nesse esquema, melhorando a qualidade do grafo e as respostas específicas do domínio.

3. **Adicione um ciclo de feedback**: Permita que o Cognee ajuste os pesos das arestas do grafo com base no feedback real dos utilizadores, para que a recuperação melhore ao longo do tempo em vez de permanecer estática.

4. **Ajuste para personalização e comportamento de sessão**: Utilize IDs de utilizador, inquilinos e conjuntos de dados para oferecer a cada pessoa ou equipa a sua própria visão sobre o motor de memória partilhado.

5. **Expanda para agentes mais complexos**: Integre o Cognee em frameworks de agentes para construir sistemas multiagente que partilhem a mesma camada de memória. *O plugin Microsoft Agent Framework x Cognee está a caminho.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
